In [1]:
import pandas as pd
import numpy as np
import random
import datetime
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_pickle('data/merged.pickle')

In [3]:
clean = data
data.head()

,session_id,time,user_id,product_id,event,discount,purchase_id,made_purchase,known_product,product,...,duration,syntetic_duration,agg_time_spent,weekday,hour,weekday_sin,weekday_cos,hour_sin,hour_cos,ends_with_purchase
0,100001.0,2020-01-17 16:08:57,102.0,1001.0,view_product,0,NaN,False,True,telefon siemens gigaset da310,...,00:02:43.251191,True,00:08:06.502383,4,16,-0.433884,-0.900969,-0.866025,-0.500000,False
1,100002.0,2020-01-07 05:10:41,102.0,1277.0,view_product,20,NaN,False,True,apple ipad mini 64gb 4g,...,00:03:45,False,00:05:23.251191,1,5,0.781831,0.623490,0.965926,0.258819,False
2,100002.0,2020-01-07 05:14:26,102.0,1276.0,view_product,20,NaN,False,True,apple ipad mini 64gb,...,00:02:43.251191,True,00:03:45,1,5,0.781831,0.623490,0.965926,0.258819,False
3,100003.0,2020-01-16 09:37:42,102.0,1276.0,view_product,0,NaN,False,True,apple ipad mini 64gb,...,00:00:40,False,00:06:28.251191,3,9,0.433884,-0.900969,0.707107,-0.707107,False
4,100003.0,2020-01-16 09:38:22,102.0,1277.0,view_product,0,NaN,False,True,apple ipad mini 64gb 4g,...,00:02:43.251191,True,00:09:08.251191,3,9,0.433884,-0.900969,0.707107,-0.707107,False


In [4]:
clean.columns

Index(['session_id', 'time', 'user_id', 'product_id', 'event', 'discount',
       'purchase_id', 'made_purchase', 'known_product', 'product', 'price',
       'cat_0', 'cat_1', 'cat_2', 'cat_3', 'city', 'street', 'name_1',
       'name_2', 'surname', 'female', 'purchase_time', 'delivery_time',
       'delivery_company', 'delivery_duration', 'duration',
       'syntetic_duration', 'agg_time_spent', 'weekday', 'hour', 'weekday_sin',
       'weekday_cos', 'hour_sin', 'hour_cos', 'ends_with_purchase'],
      dtype='object')

# Group events in sessions

In [8]:
session_grouped = clean.groupby('session_id')
session_grouped.head()

,session_id,time,user_id,product_id,event,discount,purchase_id,made_purchase,known_product,product,...,duration,syntetic_duration,agg_time_spent,weekday,hour,weekday_sin,weekday_cos,hour_sin,hour_cos,ends_with_purchase
0,100001.0,2020-01-17 16:08:57,102.0,1001.0,view_product,0,NaN,False,True,telefon siemens gigaset da310,...,00:02:43.251191,True,00:08:06.502383,4,16,-0.433884,-0.900969,-0.866025,-0.500000,False
1,100002.0,2020-01-07 05:10:41,102.0,1277.0,view_product,20,NaN,False,True,apple ipad mini 64gb 4g,...,00:03:45,False,00:05:23.251191,1,5,0.781831,0.623490,0.965926,0.258819,False
2,100002.0,2020-01-07 05:14:26,102.0,1276.0,view_product,20,NaN,False,True,apple ipad mini 64gb,...,00:02:43.251191,True,00:03:45,1,5,0.781831,0.623490,0.965926,0.258819,False
3,100003.0,2020-01-16 09:37:42,102.0,1276.0,view_product,0,NaN,False,True,apple ipad mini 64gb,...,00:00:40,False,00:06:28.251191,3,9,0.433884,-0.900969,0.707107,-0.707107,False
4,100003.0,2020-01-16 09:38:22,102.0,1277.0,view_product,0,NaN,False,True,apple ipad mini 64gb 4g,...,00:02:43.251191,True,00:09:08.251191,3,9,0.433884,-0.900969,0.707107,-0.707107,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31018,110074.0,2020-02-03 05:02:30,301.0,1050.0,view_product,0,NaN,False,True,bioshock 2 (pc),...,00:02:15,False,00:05:40,0,5,0.000000,1.000000,0.965926,0.258819,False
31019,110074.0,2020-02-03 05:04:46,301.0,1051.0,view_product,0,NaN,False,True,spec ops the line (pc),...,00:02:43.251191,True,00:00:00,0,5,0.000000,1.000000,0.965926,0.258819,False
31020,110075.0,2020-03-18 07:05:52,301.0,1278.0,view_product,5,NaN,False,True,intenso music walker 8gb,...,00:02:43.251191,True,00:06:18.251191,2,7,0.974928,-0.222521,0.965926,-0.258819,False
31021,110076.0,2020-02-01 04:19:13,301.0,1001.0,view_product,10,NaN,False,True,telefon siemens gigaset da310,...,00:01:45,False,00:00:00,5,4,-0.974928,-0.222521,0.866025,0.500000,True


In [9]:
features = pd.DataFrame(index=clean.session_id.drop_duplicates())
more = pd.DataFrame(index=clean.index)
sessions_in_time = clean.set_index(['session_id', 'time'])

In [10]:
def events_up_to(row, func):
    events = sessions_in_time.loc[row.session_id]
    return func(events[events.index <= row.time])

In [11]:
products_bought = session_grouped['made_purchase'].apply(list).apply(sum)
features['products_bought'] = products_bought
products_bought.head()

session_id
100001.0    0
100002.0    0
100003.0    0
100004.0    0
100005.0    1
Name: made_purchase, dtype: int64

In [12]:
made_purchase = session_grouped['made_purchase'].any()
more['made_purchase'] = clean.session_id.apply(lambda x: made_purchase[x])

In [13]:
more['events_count'] = clean.apply(lambda row: events_up_to(row, len), axis=1)

In [14]:
def count_products_viewed(df):
    return len(df[~df.made_purchase])

more['views'] = clean.apply(lambda row: events_up_to(row, count_products_viewed), axis=1)

In [15]:
def total_duration(df):
    return df.agg_time_spent.max().total_seconds()

more['total_duration'] = clean.apply(lambda row: events_up_to(row, total_duration), axis=1)

In [16]:
def mean_duration(df):
    return df.duration.mean().total_seconds()

more['mean_duration'] = clean.apply(lambda row: events_up_to(row, mean_duration), axis=1)

In [17]:
def products_viewed(df):
    return len(df.product_id.unique())

more['products_viewed'] = clean.apply(lambda row: events_up_to(row, products_viewed), axis=1)

In [18]:
def mean_time_per_product(df):
    return df.groupby('product_id').duration.sum().mean().total_seconds()

more['mean_time_per_product'] = clean.apply(lambda row: events_up_to(row, mean_time_per_product), axis=1)

In [19]:
more['user_id'] = clean.user_id

In [20]:
more['female'] = clean.female

In [21]:
def product_ids(df):
    return set(df.product_id.unique())

more['product_ids'] = clean.apply(lambda row: events_up_to(row, product_ids), axis=1)

In [22]:
more['weekday_cos'] = clean.weekday_cos
more['weekday_sin'] = clean.weekday_sin

In [23]:
more['hour_cos'] = clean.hour_cos
more['hour_sin'] = clean.hour_sin

In [24]:
def unique_categories(df, i):
    return set(df[f'cat_{i}'].unique())

for i in range(4):
    more[f'cat_{i}'] = clean.apply(lambda row: events_up_to(row, lambda df: unique_categories(df, i)), axis=1)

In [25]:
def any_syntetic_duration(df):
    return df.syntetic_duration.any()

more['syntetic_duration'] = clean.apply(lambda row: events_up_to(row, any_syntetic_duration), axis=1)

In [26]:
delivery_duration = session_grouped['delivery_duration'].apply(np.mean).apply(lambda x: x.total_seconds())
features['delivery_duration'] = delivery_duration
features['syntetic_delivery_duration'] = delivery_duration.isna()
mean = delivery_duration.mean()
features['delivery_duration'] = delivery_duration.fillna(value=mean)
features[['delivery_duration', 'syntetic_delivery_duration']].head()

,delivery_duration,syntetic_delivery_duration
session_id,,
100001.0,302902.182931,True
100002.0,302902.182931,True
100003.0,302902.182931,True
100004.0,302902.182931,True
100005.0,345763.000000,False


In [32]:
more.shape

(29494, 19)

In [28]:
more.to_pickle('data/features.pickle')
develop, prod = train_test_split(more, test_size=0.2)
develop.to_pickle('data/features_develop.pickle')
prod.to_pickle('data/features_prod.pickle')